# Analiza danych przestrzennych - ćwiczenia laboratoryjne 2022/2023

Ten notatnik zalicza się do grupy zestawów zadań, na podstawie których odbywa się zaliczenie ćwiczeń i podlega zwrotowi do oceny w ustalonym na zajęciach terminie.

Uwagi ogólne:
- Podczas wykonywania zadań należy korzystać wyłącznie z pakietów zaimportowanych na początku notatnika oraz z pakietów wchodzących w skład standardowej biblioteki Pythona, które można zaimportować samodzielnie we wskazanej komórce.
- Swoje rozwiązania należy wprowadzać wyłącznie w miejce następujących fragmentów kodu:<br/> ` # YOUR CODE HERE`<br/> ` raise NotImplementedError()`<br/> Nie należy w żaden sposób modyfikować pozostałych fragmentów kodu oraz elementów notatnika, w szczególności dodawać lub usuwać komórek oraz zmieniać nazwy pliku.
- Jeżeli zestaw zadań wymaga skorzystania z funkcji przygotowanych w ramach wcześniejszych zestawów zadań należy je umieścić we wskazanej komórce.
- Wszystkie wykresy powinny być wykonane w jednolitym, przejrzystym i czytelnym stylu, mieć nadane tytuły, opisane osie oraz odpowiednio dobrany rozmiar, wielkość punktów i grubość linii. Proporcje osi wykresów przedstawiających rozkłady punktów powinny być dobrane tak, aby wykresy odzwierciedlały rzeczywisty rozkład punktów w przestrzeni.
- Zadania, które powodują wyświetlenie komunikatu o błędzie przerywającym wykonywanie kodu nie podlegają ocenie.

Przed odesłaniem zestawu zadań do oceny proszę uzupełnić komórkę z danymi autorów rozwiązania (`NAME` - nazwa grupy, `COLLABORATORS` - imiona, nazwiska i numery indeksów członków grupy) oraz upewnić się, że notatnik działa zgodnie z oczekiwaniami. W tym celu należy skorzystać z opcji **Restart Kernel and Run All Cells...** dostępnej na górnej belce notatnika pod symbolem $\blacktriangleright\blacktriangleright$. 

---

## Zestaw zadań 3: Badanie intensywności procesów punktowych (część 2)

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Miejsce do importu pakietów wchodzących w skład standardowej biblioteki Pythona oraz ustawienie opcji wykorzystywanych pakietów
sns.set() 
sns.set_theme(style="whitegrid")

In [3]:
# Miejsce do wklejenie funkcji ze wcześniejszych zestawów zadań
def homogeneous_poisson_on_rectangle(intensity: float, x_lim: list, y_lim: list) -> pd.DataFrame:
    """
    Parameters
    -------
    intensity: float
        Liczba dodatnia określająca intensywność procesu punktowego.
    x_lim: list
        Lista określająca zakres wartości współrzędnej X.
        Przykład: [0, 10]
    y_lim: list
        Lista określająca zakres wartości współrzędnej Y.
        Przykład: [0, 10]   
    
    Returns
    -------
    points: DataFrame
        Tablica zawierająca dwie kolumny ze współrzędnymi punktów opisane jako "X" i "Y".
    """
    a = (x_lim[1] - x_lim[0]) * (y_lim[1] - y_lim[0])
    n = np.random.poisson(intensity * a, 1)[0]
    df = []
    
    for nn in range(n):
        df.append([np.random.uniform(x_lim[0], x_lim[1]), np.random.uniform(y_lim[0], y_lim[1])])
        
    return pd.DataFrame(data=df, columns=["X", "Y"])

def point_count_on_subregions(points: pd.DataFrame, bins: list, x_lim: list, y_lim: list) -> list:
    """
    Parameters
    -------
    points: DataFrame
        Tablica zawierająca dwie kolumny ze współrzędnymi punktów opisane jako "X" i "Y".
    bins: list
        Lista określająca liczbę podobszarów w poziomie i pionie.
        Przykład: [10, 10]
    x_lim: list
        Lista określająca zakres wartości współrzędnej X.
        Przykład: [0, 10]
    y_lim: list
        Lista określająca zakres wartości współrzędnej Y.
        Przykład: [0, 10]   

    Returns
    -------
    bin_data: list
        Lista zawierająca trzy macierze:
        - 1D ze współrzędnymi krawędzi podobszarów na osi X,
        - 1D ze współrzędnymi krawędzi podobszarów na osi Y,
        - 2D z liczbą punków przypisanych do każdego z podobszarów.
        Na przykład: [array([0, 1, 2]), array([0, 1, 2]), array([[7, 2], [4, 5]])]
    """
    num_of_points, x_edges, y_edges = np.histogram2d(points["X"], points["Y"], bins, [x_lim, y_lim])
    return [x_edges, y_edges, num_of_points.T]


### Przygotowanie danych
Wczytaj dane zawarte w plikach CSV załączonych do zestawu zadań.

In [4]:
data1 = pd.read_csv("data/points_1.csv")
data2 = pd.read_csv("data/points_2.csv")
data3 = pd.read_csv("data/points_3.csv")

### Zadanie 1: Test chi-kwadrat Pearsona (30 pkt)

Przygotuj funkcję `pearsons_chi2_test()`, która będzie przeprowadzać test istotności chi-kwadrat Pearsona i wyświetlać jego wynik zgodnie z pokazanym poniżej schematem. Następnie wykorzystaj przygotowanią funkcję do sprawdzenia, czy rozkłady punktowe zaimportowane z plików points_1.csv i points_2.csv są jednorodnymi rozkładami Poissona o intensywności równej 20. W obliczeniach przyjmij $\alpha=0.05$.

Rozwiązanie zadania wymaga dodatkowo przygotowania funkcji pomocniczych `distribution_table()` i `poisson_distribution_table()`, które będą przygotowywać szeregi rodzielcze testowanego rozkładu oraz teoretycznego rozkładu Poissona.

Algorytm postępowania:
- Formułujemy hipotezę zerową i hipotezę alternatywną H1: <br/>
H0: Testowana zmienna ma przyjęty rozkład teoretyczny <br/>
H1: Testowana zmienna nie ma przyjętego rozkładu teoretycznego
- Obliczamy wartość statystyki testowej $\chi^2$: <br/>
$\chi^2 = \sum_{i=1}^{k} \frac{(n_i-n p_i)^2}{np_i}$ <br/>
gdzie: $k$ - liczba wariantów badanej cechy, $n_i$ - liczebność i-tego wariantu testowanego rozkładu, $n$ - liczba punktów testowanego rozkładu, $p_i$ - prawdopodobieństwo  i-tego wariantu rozkładu teoretycznego.
- Z rozkładu chi-kwadrat wyznaczamy obszar krytyczny testu istotności $\chi^2_{\alpha}$: <br/>
$\chi^2_{\alpha} = \chi^2_{1-\alpha, k-s-1}$ <br/>
gdzie:  $\alpha$ - poziom istotności, $k$ - liczba wariantów rozkładu, $s$ - liczba nieznanych parametrów rozkładu.
- Podejmujemy decyzję weryfikującą: <br/>
$\chi^2 >= \chi^2_{\alpha}$ - Odrzucenie H0 na rzecz H1 na poziomie istotności alpha = X <br/>
$\chi^2 < \chi^2_{\alpha}$ - Wynik testu istotności nie daje podstaw do odrzucenia H0 na rzecz H1 na poziomie istotności alpha = X

Przykładowe wyniki pracy funkcji `pearsons_chi2_test()`: <br/>
<br/>
`Test chi-kwadrat Pearsona` <br/>
`H0: Testowana zmienna ma przyjęty rozkład teoretyczny` <br/>
`H1: Testowana zmienna nie ma przyjętego rozkładu teoretycznego` <br/>
`chi2 = 23.307 chi2_alpha = 18.078`<br/>
`chi2 >= chi2_alpha` <br/>
`Odrzucenie H0 na rzecz H1 na poziomie istotności alpha = 0.05` <br/>
<br/>
`Test chi-kwadrat Pearsona` <br/>
`H0: Testowana zmienna ma przyjęty rozkład teoretyczny` <br/>
`H1: Testowana zmienna nie ma przyjętego rozkładu teoretycznego` <br/>
`chi2 = 19.521 chi2_alpha = 21.129`<br/>
`chi2 < chi2_alpha` <br/>
`Wynik testu istotności nie daje podstaw do odrzucenia H0 na rzecz H1 na poziomie istotności alpha = 0.05` <br/>

#### a) Przygotowanie funkcji

In [5]:
def distribution_table(bin_counts: np.array) -> pd.DataFrame:
    """
    Parameters
    -------
    bin_counts: array
        Macierz 2D z liczbą punków przypisanych do każdego z podobszarów.

    Returns
    -------
    table: DataFrame
        Tablica zawierająca 2 kolumny:
        - "K", która zawiera wszystkie wartości całkowite z zakresu od minimalnej do maksymalnej liczby zliczeń w obrębie podobszarów,
        - "N(K)", która zawiera liczby podobszarów, którym zostały przypisane poszczególne liczby punktów.
    """    
    minimum = int(np.amin(bin_counts))
    maximum = int(np.amax(bin_counts))
    K = list(range(minimum, maximum+1))
    N = [np.count_nonzero(bin_counts==k) for k in K]
    return pd.DataFrame({"K": K, "N(K)": N})

def poisson_distribution_table(k: np.array, mu: int) -> pd.DataFrame:
    """
    Parameters
    -------
    k: array
        Macierz 1D z wariantami badanej cechy, dla którym ma zostać wyliczone prawdopodobieństwo.
    mu: int
        Wartość oczekiwana rozkładu Poissona.

    Returns
    -------
    table: DataFrame
        Tablica zawierająca 2 kolumny:
        - "K", która zawiera warianty badanej cechy,
        - "P(K)", która zawiera wartości prawdopodobieństw rozkładu Poissona wyliczone dla wartości oczekiwanej mu
        oraz poszczególnych wariantów badanej cechy znormalizowane do sumy wartości równej 1.
    """  
    probabilities = [sp.stats.poisson.pmf(w,mu) for w in k]
    return pd.DataFrame({"K": k, "P(K)": probabilities/sum(probabilities)})

def pearsons_chi2_test(tested_distribution: pd.DataFrame, theoretical_distribution: pd.DataFrame,
                       alpha: float, ddof: int) -> None:
    """
    Parameters
    -------
    tested_distribution: DataFrame
        Tablica opisująca testowany rozkład i zawierająca 2 kolumny:
        - "K", która zawiera warianty badanej cechy, wartości muszą być identycznej jak kolumna "K" zmiennej lokalnej theoretical_distribution,
        - "N(K)", która zawiera liczebności poszczególnych wariantów badanej cechy.

    theoretical_distribution: DataFrame
        Tablica opisująca rozkład teoretyczny i zawierająca 2 kolumny:
        - "K", która zawiera warianty badanej cechy, wartości muszą być identycznej jak kolumna "K" zmiennej lokalnej tested_distribution,
        - "P(K)", która zawiera prawdopodobieństwa poszczególnych wariantów badanej cechy. Wartości z tej kolumny muszą sumować się do 1.
    
    alpha: float
        Wartość z zakresu [0,1] określająca poziom istotności.
    
    ddof: int
        Liczba nieujemna określająca liczbę nieznanych parametrów rozkładu.
    """
    print("Test chi-kwadrat Pearsona\nH0: Testowana zmienna ma przyjęty rozkład teoretyczny\nH1: Testowana zmienna nie ma przyjętego rozkładu teoretycznego")
    n = sum(tested_distribution["N(K)"])
    chi2 = 0
    chi2 = np.sum((tested_distribution["N(K)"]-n*theoretical_distribution["P(K)"])**2/(n*theoretical_distribution["P(K)"]))
    chi2_alpha = sp.stats.chi2.ppf(1-alpha, len(tested_distribution["K"])-ddof-1)
    print(f"chi2 = {chi2:.3f} chi2_alpha = {chi2_alpha:.3f}")
    if chi2 < chi2_alpha:
        print("chi2 < chi2_alpha")
        print(f"Wynik testu istotności nie daje podstaw do odrzucenia H0 na rzecz H1 na poziomie istotności alpha = {alpha}")
        return
    print("chi2 >= chi2_alpha")
    print(f"Wynik testu istotności daje podstawy do odrzucenia H0 na rzecz H1 na poziomie istotności alpha = {alpha}")
    

#### b) Weryfikacja hipotezy o rozkładzie 1

In [6]:
bins = [40, 20]
area = 20 * 10 / (bins[0] * bins[1])
intensity = 20
alpha = 0.05
*_, binned = point_count_on_subregions(data1, bins, [0,20], [0,10])
distribution = distribution_table(binned)
theoretical = poisson_distribution_table(distribution["K"], intensity * area)
pearsons_chi2_test(distribution, theoretical, alpha, 0)

Test chi-kwadrat Pearsona
H0: Testowana zmienna ma przyjęty rozkład teoretyczny
H1: Testowana zmienna nie ma przyjętego rozkładu teoretycznego
chi2 = 58802.637 chi2_alpha = 11.070
chi2 >= chi2_alpha
Wynik testu istotności daje podstawy do odrzucenia H0 na rzecz H1 na poziomie istotności alpha = 0.05


#### c) Weryfikacja hipotezy o rozkładzie 2

In [7]:
*_, binned = point_count_on_subregions(data2, bins, [0,20], [0,10])
distribution = distribution_table(binned)
theoretical = poisson_distribution_table(distribution["K"], intensity * area)
pearsons_chi2_test(distribution, theoretical, alpha, 0)

Test chi-kwadrat Pearsona
H0: Testowana zmienna ma przyjęty rozkład teoretyczny
H1: Testowana zmienna nie ma przyjętego rozkładu teoretycznego
chi2 = 21733.970 chi2_alpha = 16.919
chi2 >= chi2_alpha
Wynik testu istotności daje podstawy do odrzucenia H0 na rzecz H1 na poziomie istotności alpha = 0.05


### Zadanie 2: Test Kołmogorowa - Smirnowa (20 pkt)

Przygotuj funkcję `kolmogorow_smirnow_test()`, która będzie przeprowadzać test istotności Kołmogorowa-Smirnowa i wyświetlać jego wynik zgodnie z pokazanym poniżej schematem. Następnie wykorzystaj przygotowanią funkcję do sprawdzenia, czy rozkład punktowy zaimportowany z pliku points_3.csv jest jednorodnym rozkładem Poissona. W obliczeniach przyjmij poziom istotności $\alpha=0.05$.

Algorytm postępowania:
- Formułujemy hipotezę zerową i hipotezę alternatywną H1: <br/>
H0: Testowana zmienna ma przyjęty rozkład teoretyczny <br/>
H1: Testowana zmienna nie ma przyjętego rozkładu teoretycznego
- Obliczamy wartość statystyki testowej $\lambda$: <br/>
$D = \sup_{x}(|F_t - F_0|)$ <br/>
$\lambda = D\sqrt{n}$ <br/>
gdzie: $F_t$ - dystrybuanta testowanego rozkładu,  $F_0$ - dystrybuanta rozkładu teoretycznego, $n$ - liczba punktów.
- Z rozkładu Kołomogorowa wyznaczamy obszar krytyczny testu istotności $\lambda_{\alpha}$: <br/>
$\lambda_{\alpha} = \lambda_{1-\alpha}$ <br/>
gdzie:  $\alpha$ - poziom istotności.
- Podejmujemy decyzję weryfikującą: <br/>
$\lambda >= \lambda_{\alpha}$ - Odrzucenie H0 na rzecz H1 na poziomie istotności alpha = X <br/>
$\lambda < \lambda_{\alpha}$ - Wynik testu istotności nie daje podstaw do odrzucenia H0 na rzecz H1 na poziomie istotności alpha = X

Uwaga! Test należy przeprowadzić niezależnie dla współrzędnej X i Y. Decyzja jest podejmowana na podstawie wyników obu testów.


Przykładowe wyniki pracy funkcji `kolmogorow_smirnow_test()`: <br/>
<br/>
`Test Kołmogorowa-Smirnowa dla współrzędnej X` <br/>
`H0: Testowana zmienna ma przyjęty rozkład teoretyczny` <br/>
`H1: Testowana zmienna nie ma przyjętego rozkładu teoretycznego` <br/>
`lambda = 2.036  lambda_alpha = 1.255`<br/>
`lambda >= lambda_alpha` <br/>
`Odrzucenie H0 na rzecz H1 na poziomie istotności alpha = 0.05` <br/>
<br/>
`Test Kołmogorowa-Smirnowa dla współrzędnej Y` <br/>
`H0: Testowana zmienna ma przyjęty rozkład teoretyczny` <br/>
`H1: Testowana zmienna nie ma przyjętego rozkładu teoretycznego` <br/>
`lambda = 1.136  lambda_alpha = 1.748` <br/>
`lambda < D_alpha` <br/>
`Wynik testu istotności nie daje podstaw do odrzucenia H0 na rzecz H1 na poziomie istotności alpha = 0.05`

#### a) Przygotowanie funkcji

In [8]:
def kołmogorow_smirnow_test(tested_points: pd.DataFrame, theoretical_points: pd.DataFrame,
                            alpha: float, ddof: int) -> None:
    """
    Parameters
    -------
    tested_points: DataFrame
        Tablica zawierająca kolumnę ze współrzędnymi punktów testowanego rozkładu opisaną jako "X" lub "Y".

    theoretical_points: DataFrame
        Tablica zawierająca kolumnę ze współrzędnymi punktów toeretycznego rozkładu opisaną jako "X" lub "Y".
    
    alpha: float
        Wartość z zakresu [0,1] określająca poziom istotności.
    
    ddof: int
        Liczba nieujemna określająca liczbę nieznanych parametrów rozkładu.
    """
    
    def kołmogorow_smirnow_test__for_one_coord(tested, theoretical, a, colname, ddof = 0):
        print(f"Test Kołomgorova-Smirnova dla współrzędnej {colname}\nH0: Testowana zmienna ma przyjęty rozkład teoretyczny\nH1: Testowana zmienna nie ma przyjętego rozkładu teoretycznego")
        n = len(tested_points[colname])
        ks, _ = sp.stats.kstest(tested_points[colname], theoretical_points[colname])
        lbda = n ** 0.5 * ks
        lbda_alpha = sp.stats.kstwobign.ppf(1 - alpha)
        print(f"lambda = {lbda:.3f} lambda_alpha = {lbda_alpha:.3f}")
        if lbda < lbda_alpha:
            print("lambda < D_alpha")
            print(f"Wynik testu istotności nie daje podstaw do odrzucenia H0 na rzecz H1 na poziomie istotności alpha = {alpha}\n")
        else:
            print("lambda >= lambda_alpha")
            print(f"Odrzucenie H0 na rzecz H1 na poziomie istotności alpha = {alpha}\n")

    kołmogorow_smirnow_test__for_one_coord(tested_points, theoretical_points, alpha, "X", ddof)
    kołmogorow_smirnow_test__for_one_coord(tested_points, theoretical_points, alpha, "Y", ddof)             
    

#### b) Weryfikacja hipotezy o rozkładzie 3

In [9]:
theoretical_data = homogeneous_poisson_on_rectangle(20, [20,0], [10,0])
kołmogorow_smirnow_test(data3, theoretical_data, 0.05, 0)

Test Kołomgorova-Smirnova dla współrzędnej X
H0: Testowana zmienna ma przyjęty rozkład teoretyczny
H1: Testowana zmienna nie ma przyjętego rozkładu teoretycznego
lambda = 1.069 lambda_alpha = 1.358
lambda < D_alpha
Wynik testu istotności nie daje podstaw do odrzucenia H0 na rzecz H1 na poziomie istotności alpha = 0.05

Test Kołomgorova-Smirnova dla współrzędnej Y
H0: Testowana zmienna ma przyjęty rozkład teoretyczny
H1: Testowana zmienna nie ma przyjętego rozkładu teoretycznego
lambda = 1.146 lambda_alpha = 1.358
lambda < D_alpha
Wynik testu istotności nie daje podstaw do odrzucenia H0 na rzecz H1 na poziomie istotności alpha = 0.05

